In [1]:
import pandas as pd
import numpy as np

import rdkit
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem import Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
enumerator = rdMolStandardize.TautomerEnumerator()

from rdkit.Chem import PandasTools

import warnings
warnings.filterwarnings('ignore')

In [12]:
#lit=pd.read_excel('Updated_AnaerobicRxn_LitLib_04102024.xlsx')
lit=pd.read_excel('Updated_AnaerobicRxn_LitLib_04242024.xlsx')
#newEx=pd.read_excel('AdditionalExamples_carbamateOpesterCae.xlsx', sheet_name='CombinedData')

7: < 30 min OR 0.0208333 day
6: 30 min - 200 min 0.0208333 - 0.138889 day
5: 200 min - 1 day OR 0.138889 to 1 day
4: 1 day- 7 days
3: 7 days to 60 days
2: 60 days to 365 days
1 : > 365 days or stable

In [13]:
for i in lit.index:
    hl=lit['HalfLife (days)'][i]
    if hl == 'stable':
        lit['Rank'][i]=1
    else:
        hl=float(lit['HalfLife (days)'][i])
        if hl < 0.0208333:
            lit['Rank'][i]=7
        elif 0.0208333 <= hl < 0.138889:
            lit['Rank'][i]=6
        elif 0.138889 <= hl < 1:
            lit['Rank'][i]=5
        elif 1 <= hl < 7:
            lit['Rank'][i]=4
        elif 7 <= hl < 60:
            lit['Rank'][i]=3
        elif 60 <= hl <= 365:
            lit['Rank'][i]=2
        elif hl > 365:
            lit['Rank'][i]=1

In [14]:
#lit.to_excel('Updated_AnaerobicRxn_LitLib_04122024.xlsx',index=False)

In [15]:
#remove data rows with not rate data
lit=lit.replace('nan', np.nan)
sub=lit.dropna(axis=0,subset ='HalfLife (days)')
sub

,Scheme,Rxn_Type,Rxn,Rxn_Scheme,Redox,LibDoc_Check,LibDoc_Check_Notes,Parent,Parent_DTXSID,Parent_CASRN,...,HalfLife (days),Rank,RateConstant_avg,RateConstant_avg2,QA_Check,QA_Revisions,QA_Rate,QA_HalfLife,Reference,DOI
1,Biodegradation of Saturated Alkanes,Addition,Methylbenzene to fumarate adduct,Addition: Methylbenzene to fumarate adduct,sulfate reducing,Agree,under Biodegradation of saturate alkane schem...,Toluene,DTXSID7021360,108-88-3,...,124.74,2.0,NaN,NaN,Agree;Fig2,NaN,NaN,0.005556,"Beller and Spormann, 1997b",10.1128/aem.63.9.3729-3731.1997
2,Biodegradation of Saturated Alkanes,Addition,Methylbenzene to fumarate adduct,Addition: Methylbenzene to fumarate adduct,sulfate reducing,NaN,NaN,o-Xylene,DTXSID3021807,95-47-6,...,15.932853,3.0,NaN,NaN,Agree; Fig. 4,NaN,NaN,NaN,"Beller et al., 1996",10.1128/aem.62.4.1188-1196.1996
3,Biodegradation of Saturated Alkanes,Addition,Methylbenzene to fumarate adduct,Addition: Methylbenzene to fumarate adduct,nitrate reducing,Agree,under Biodegradation of saturate alkane schem...,Toluene,DTXSID7021360,108-88-3,...,0.009625,7.0,NaN,NaN,"Fig 6, screenshort of fig 6 is in SI_Anaerobic...","Agree; Results --""Benzylsuccinate formation. I...",NaN,NaN,"Biegert et al., 1996",10.1111/j.1432-1033.1996.0661w.x
4,Biodegradation of Saturated Alkanes,Addition,Methylbenzene to fumarate adduct,Addition: Methylbenzene to fumarate adduct,iron reducing,Agree,under Biodegradation of saturated alkane sche...,m-Xylene,DTXSID6026298,108-38-3,...,3.3,4.0,NaN,NaN,Fig 1g ; cant find calculations to confirm,"Agree; Table 1 and fig 2e -- no calculations, ...",0.3,NaN,"Botton and Parsons., 2006",10.1007/s10532-006-9071-9
7,Biodegradation of Saturated Alkanes,Activation,Methylbenzene to fumarate adduct,Addition: Methylbenzene to fumarate adduct,sulfate reducing,Agree,under Biodegredation of saturated alkanes sch...,M-cresol,DTXSID6024200,108-39-4,...,0.022071,6.0,NaN,NaN,Agree; Fig 2; SI_AnaerobicBioData,NaN,NaN,NaN,"Muller et al., 1999",10.1007/s002030050782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,NaN,NaN,NaN,Biotic Hydrolysis: Carbamates,NaN,NaN,NaN,Carbaryl,DTXSID9020247,NaN,...,6.2,4.0,NaN,NaN,Table 1. (Metabolism),NaN,NaN,NaN,"Liu et la., 2000",10.1002/1522-7278(2000)15:5<476::AID-TOX16>3.0...
1149,NaN,NaN,NaN,Biotic Hydrolysis: Carbamates,NaN,NaN,NaN,Carbaryl,DTXSID9020247,NaN,...,13.5,3.0,NaN,NaN,Reported half life is average of half life ran...,NaN,NaN,NaN,"Gunasekara, 2007",https://citeseerx.ist.psu.edu/document?repid=r...
1150,NaN,NaN,NaN,Biotic Hydrolysis: Carbamates,NaN,NaN,NaN,Carbofuran,NaN,NaN,...,23.4,3.0,NaN,NaN,Transformation shown in Fig. 11; Calculations ...,NaN,NaN,NaN,"Krishna and Philip, 2008",10.1080/03601230701795155
1151,NaN,NaN,NaN,Biotic Hydrolysis: Carbamates,NaN,NaN,NaN,Methomyl,DTXSID1022267,NaN,...,0.041667,6.0,NaN,NaN,Table 4; only reporting value from Emmercompas...,NaN,NaN,NaN,"Browmilow et al., 1985",10.1002/ps.2780170512


In [16]:
#subset rxn, hl and rank
#rankTable=lit[['Rxn_Type','Rxn','Parent','Parent_SMILES','Product_SMILES','HalfLife (days)','Rank']]
rankTable=sub[['Rxn_Scheme','HalfLife (days)']]


In [17]:
# remove 'stable' data points for now so that I can get median hl
new=rankTable[rankTable['HalfLife (days)']!='stable']
print(new['Rxn_Scheme'].value_counts())
new=new.groupby('Rxn_Scheme', group_keys= True)[['HalfLife (days)']].median().reset_index()
new.rename(columns={'HalfLife (days)':'Med_HalfLife(days)'},inplace=True)

Rxn_Scheme
Addition: Methylbenzene to fumarate adduct                          190
Reduction: Dehalogenation of aromatic halides                        70
Hydroxylation: Alkyl benzene to sec-hydroxy benzene                  48
Carboxylation: Substituted benzene to substituted benzoic acid       23
Reduction: Hydrogenolysis of haloalkanes                             23
                                                                   ... 
Reduction: Sulfoxide to sulfide                                       1
Reduction: Quinone reduction                                          1
Carboxylation: Phenyl acetate to phenyl keto carboxylic acid          1
Reduction: Dehydroxylation of 2,5-dihydroxybenzoyl-CoA                1
Nucleophilic Substitution: Sulfide substitution of alkyl halides      1
Name: count, Length: 87, dtype: int64


In [18]:
rank=[]
for i in new.index:
    hl=new['Med_HalfLife(days)'][i]
    hl=float(new['Med_HalfLife(days)'][i])
    if hl < 0.0208333:
        rank.append(7)
    elif 0.0208333 <= hl < 0.138889:
        rank.append(6)
    elif 0.138889 <= hl < 1:
        rank.append(5)
    elif 1 <= hl < 7:
        rank.append(4)
    elif 7 <= hl < 60:
        rank.append(3)
    elif 60 <= hl <= 365:
        rank.append(2)
    elif hl > 365:
        rank.append(1)
new['Rank']=rank
new

,Rxn_Scheme,Med_HalfLife(days),Rank
0,Activation: Benzylsuccinic acid to benzylsucci...,0.022299,6
1,Activation: CoA addition to alpha-branched fat...,4.442308,4
2,Activation: CoA addition to beta-branched fatt...,2.313529,4
3,Activation: CoA addition to ketone,0.000058,7
4,Activation: CoA addition to phenanthrene-2-car...,31.187142,3
...,...,...,...
82,Reduction: Sulfoxide to sulfide,6.3,4
83,Reduction: Vicinal dehalogenation,33.216226,3
84,"Ring Closing: 1,2-phenylenediamine to benzodia...",33.4,3
85,"Ring Opening: 1,3-Cyclohexadione",0.031885,6


In [19]:
#what is in new that is not in schemes
list(set(list(new['Rxn_Scheme'])) - set(list(schemes)))

[]

In [20]:
#what is in schemes that is not in new (i.e what schemes do not have rate data)
print(len(list(set(schemes)-set(list(new['Rxn_Scheme'])))))
list(set(schemes)-set(list(new['Rxn_Scheme'])))

55


['Nucleophilic Substitution: Vicinal halogenated aliphatics',
 'Oxidation: Beta-like oxidation of beta-branched fatty acyl-CoA',
 'Reduction: Terminal alkene to alkane',
 'Ring Opening: 4,4a-hexahydro-2-naphthoyl-CoA to carboxycyclohexylacetyl-CoA',
 'Oxidation: Beta oxidation of alpha-branched fatty acyl-CoA',
 'Addition: Alkane to fumarate adduct',
 'Hydroxylation: 3-Hydroxybenzoic acid to 2,5-dihydroxybenzoic acid',
 'Oxidation: Malonic acid to acetyl-CoA',
 'Activation: CoA addition to 3-oxo carboxylic acid',
 'Oxidation: Glutaryl-CoA to acetyl-CoA',
 'Thiolytic cleavage of beta-keto acyl-CoA to acyl-CoA',
 'Activation: Phenylglyoxylic acid to benzoyl-CoA',
 'Ring opening: Oxoenoyl-CoA to 3-hydroxypimelyl-CoA',
 'Decarboxylation: Rearranged fumarate adduct',
 'Oxidation: Cyclohexanoyl-CoA to cyclohexenecarboxyl-CoA',
 'Activation: CoA addition to cycloalkyl succinic acid',
 'Rearrangement: Cycloalkyl succinyl-CoA to Cycloalkyl methylmalonyl‐CoA',
 'Ring Opening: Imidazolone to form

In [21]:
#make rank table
results=pd.DataFrame()
results['Rxn_Scheme']=schemes
results=results.merge(new, how='outer', on='Rxn_Scheme')
results

,Rxn_Scheme,Med_HalfLife(days),Rank
0,Activation: CoA addition to substituted benzoate,0.368029,5.0
1,Deamination: Meta-aminobenzoyl-CoA to benzoyl-CoA,NaN,NaN
2,Deamination: Ortho-aminobenzoyl-CoA to benzoyl...,NaN,NaN
3,Deamination: Para-aminobenzoyl-CoA to benzoyl-CoA,0.116119,6.0
4,Reduction: Dehydroxylation of 3-hydroxybenzoyl...,NaN,NaN
...,...,...,...
137,Fermentation of dichloromethane,2.01093,4.0
138,"Rearrangement: 1,2,3-Trihydroxybenzene to 1,3,...",0.007349,7.0
139,Ring Closing: 1-carbonyl-phenylhydroxylamine t...,NaN,NaN
140,"Ring Closing: 1,2-phenylenediamine to benzodia...",33.4,3.0


In [22]:
#lit.to_excel('Updated_AnaerobicRxn_LitLib_04122024.xlsx',index=False)
#review.to_excel('LitLibSubset_ReviewRates.xslx',index=False)

In [22]:
#results.to_excel('AnaerobicLibrary_RankTable_04182024.xlsx', index=False)
results.to_excel('AnaerobicLibrary_RankTable_04242024.xlsx', index=False)

In [25]:
ad_par=['Acephate', 'Acetochlor', 'Atrazine', 'Bifenthrin', 'Carbaryl',
       'Chlorothalonil', 'Chlorpyrifos', 'Dicamba', 'Dichlorvos',
       'Malathion', 'Paraquat dichloride', 'Pyraclostrobin',
       'Tebuconazole', 'Triclopyr']
lit.loc[lit['Parent'].isin(ad_par)==True,'Rxn_Scheme'].unique()

array(['Dealkylation: O-dealkylation of N-heterocyclic ether',
       'Biotic Hydrolysis: Carbamates',
       'Demethoxylation: N-demethoxylation of methoxy anilines and amines',
       'Nucleophilic Substitution: Halogenated Aromatics',
       'Biotic Hydrolysis: Organophosphorus triester (neutral)',
       'Biotic Hydrolysis: Carboxylic Acid Esters', nan,
       'Biotic Hydrolysis: Organophosphorus triester (base-catalyzed)',
       'Reduction: Hydrogenolysis of haloalkanes'], dtype=object)

## Updated Rank Table 06-11-2024

In [4]:
lit=pd.read_excel('Updated_AnaerobicRxn_LitLib_06112024_parentType.xlsx')


#omit data point that were reported for having a 'stable' half life for the purpose of this analysis
lit=lit[lit['HalfLife (days)']!='stable']
#drop rows with no half life
lit=lit[lit['HalfLife (days)'].notna()]

In [5]:
RankTable=lit.groupby(['Rxn_Scheme'])['HalfLife (days)'].median().reset_index()
RankTable.rename(columns={'HalfLife (days)':'Med_HalfLife(days)'},inplace=True)

In [6]:
rank=[]
for i in RankTable.index:
    hl=RankTable['Med_HalfLife(days)'][i]
    hl=float(RankTable['Med_HalfLife(days)'][i])
    if hl < 0.0208333:
        rank.append(7)
    elif 0.0208333 <= hl < 0.138889:
        rank.append(6)
    elif 0.138889 <= hl < 1:
        rank.append(5)
    elif 1 <= hl < 7:
        rank.append(4)
    elif 7 <= hl < 60:
        rank.append(3)
    elif 60 <= hl <= 365:
        rank.append(2)
    elif hl > 365:
        rank.append(1)
RankTable['Rank']=rank
RankTable

,Rxn_Scheme,Med_HalfLife(days),Rank
0,Activation: Benzylsuccinic acid to benzylsucci...,0.022299,6
1,Activation: CoA addition to alpha-branched fat...,4.442308,4
2,Activation: CoA addition to beta-branched fatt...,2.313529,4
3,Activation: CoA addition to ketone,0.000058,7
4,Activation: CoA addition to phenanthrene-2-car...,31.187142,3
...,...,...,...
84,Reduction: Sulfoxide to sulfide,6.3,4
85,Reduction: Vicinal dehalogenation,33.216226,3
86,"Ring Closing: 1,2-phenylenediamine to benzodia...",33.4,3
87,"Ring Opening: 1,3-Cyclohexadione",0.031885,6


In [7]:
old=pd.read_excel('AnaerobicLibrary_RankTable_05022024.xlsx')
old=old[['Rxn_Scheme','AddedExamples_Med_HalfLife(days)','Assigned_Rank']]
old=old.loc[:145]

In [8]:
new=pd.merge(RankTable,old, how='left',on='Rxn_Scheme')
new=new.astype({'Assigned_Rank': 'int'})
new

,Rxn_Scheme,Med_HalfLife(days),Rank,AddedExamples_Med_HalfLife(days),Assigned_Rank
0,Activation: Benzylsuccinic acid to benzylsucci...,0.022299,6,0.022299,6
1,Activation: CoA addition to alpha-branched fat...,4.442308,4,4.442308,4
2,Activation: CoA addition to beta-branched fatt...,2.313529,4,2.313529,4
3,Activation: CoA addition to ketone,0.000058,7,0.000058,7
4,Activation: CoA addition to phenanthrene-2-car...,31.187142,3,31.187142,3
...,...,...,...,...,...
84,Reduction: Sulfoxide to sulfide,6.3,4,6.300000,4
85,Reduction: Vicinal dehalogenation,33.216226,3,33.216226,3
86,"Ring Closing: 1,2-phenylenediamine to benzodia...",33.4,3,33.400000,3
87,"Ring Opening: 1,3-Cyclohexadione",0.031885,6,0.031885,6


In [10]:
new['AddedExamples_Med_HalfLife(days)'].median()

4.461697032903846

In [4]:
df=pd.read_excel('RankTable_06132024.xlsx', sheet_name='RankTable_061424')

In [7]:
rank=[]
for i in df.index:
    hl=df['Med_HL_days'][i]
    hl=float(df['Med_HL_days'][i])
    if hl < 0.0208333:
        rank.append(7)
    elif 0.0208333 <= hl < 0.138889:
        rank.append(6)
    elif 0.138889 <= hl < 1:
        rank.append(5)
    elif 1 <= hl < 7:
        rank.append(4)
    elif 7 <= hl < 60:
        rank.append(3)
    elif 60 <= hl <= 365:
        rank.append(2)
    elif hl > 365:
        rank.append(1)
    else:
        rank.append('NA')
df['RateSupported_Rank']=rank
df

,idx,Rxn_Scheme,Med_HL_days,RateSupported_Rank,AssignedRank,Notes
0,1,Activation: CoA addition to substituted benzoate,0.368029,5,5,NaN
1,2,Deamination: Meta-aminobenzoyl-CoA to benzoyl-CoA,NaN,NA,3,NaN
2,3,Deamination: Ortho-aminobenzoyl-CoA to benzoyl...,NaN,NA,3,NaN
3,4,Deamination: Para-aminobenzoyl-CoA to benzoyl-CoA,0.116119,6,6,NaN
4,5,Reduction: Dehydroxylation of 3-hydroxybenzoyl...,NaN,NA,3,NaN
...,...,...,...,...,...,...
141,142,Fermentation of dichloromethane,2.010930,4,4,NaN
142,143,"Rearrangement: 1,2,3-Trihydroxybenzene to 1,3,...",0.007349,7,7,NaN
143,144,Ring Closing: 1-carbonyl-phenylhydroxylamine t...,NaN,NA,4,Adjusted
144,145,"Ring Closing: 1,2-phenylenediamine to benzodia...",33.400000,3,3,NaN


In [8]:
#df.to_excel('RankTale_061424.xlsx',index=False)